In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_community.document_loaders import PyPDFLoader
import os

pdf_dir = "C:\\Workspaces\\pdf"

In [6]:
# 전체 PDF파일 로드
all_documents = []
for file_name in os.listdir(pdf_dir):
    if file_name.endswith('.pdf'):  # PDF 파일만 처리
        loader = PyPDFLoader(os.path.join(pdf_dir, file_name))
        documents = loader.load()
        all_documents.extend(documents)  # 모든 문서를 리스트에 추가
        
print(f"총 {len(all_documents)}개의 문서를 로드했습니다.")

총 12178개의 문서를 로드했습니다.


### 전처리

In [8]:
# 개행문자 제거
for doc in all_documents:
    doc.page_content = doc.page_content.replace('\n', ' ')
all_documents[0]

Document(metadata={'source': 'C:\\Workspaces\\pdf\\(1808 최종) 집단적 노사관계 판례집_홈페이지 게시용.pdf', 'page': 0}, page_content='목 차 (1) 제1장 당사자 적격························································································1제1절 노동조합법상 근로자·························································································31. 노동조합법상근로자의개념·············································································32. 근로자개념의비교·····························································································33. 노동조합법상근로자성판단기준·····································································4  3.1. 기존판례의태도··························································································4  3.2. 최근판례가제시한판단기준····································································54. 판례·························································································································7  4.1 노동조합법상근로자성을긍정한사례···············································

In [9]:
from langchain.schema import Document
# 유효하지 않은 유니코드 문자 제거
def clean_text(text):
    return text.encode('utf-8', 'ignore').decode('utf-8')

# 모든 문서에 적용
cleaned_docs = [
    Document(page_content=clean_text(doc.page_content), metadata=doc.metadata)
    for doc in all_documents
]

In [10]:
# text split
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200, 
    chunk_overlap=300, 
)
docs = splitter.split_documents(cleaned_docs) # split_documents: 여러건의 문서
print(len(docs))
docs[0:5]

17325


[Document(metadata={'source': 'C:\\Workspaces\\pdf\\(1808 최종) 집단적 노사관계 판례집_홈페이지 게시용.pdf', 'page': 0}, page_content='목 차 (1) 제1장 당사자 적격························································································1제1절 노동조합법상 근로자·························································································31. 노동조합법상근로자의개념·············································································32. 근로자개념의비교·····························································································33. 노동조합법상근로자성판단기준·····································································4  3.1. 기존판례의태도··························································································4  3.2. 최근판례가제시한판단기준····································································54. 판례·························································································································7  4.1 노동조합법상근로자성을긍정한사례··············································

### 임베딩 모델 생성(청크 저장)

In [4]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model = 'text-embedding-3-small', 
)

### 벡터 스토어

In [12]:
from langchain_chroma.vectorstores import Chroma
len(docs)
print(docs[0])

page_content='목 차 (1) 제1장 당사자 적격························································································1제1절 노동조합법상 근로자·························································································31. 노동조합법상근로자의개념·············································································32. 근로자개념의비교·····························································································33. 노동조합법상근로자성판단기준·····································································4  3.1. 기존판례의태도··························································································4  3.2. 최근판례가제시한판단기준····································································54. 판례·························································································································7  4.1 노동조합법상근로자성을긍정한사례·····················································7    가. 골프장경기보조원···············································································

In [13]:
vector_store = Chroma.from_documents(docs[:1000], embeddings, persist_directory='./DB')

In [14]:
batch_size = 1000 # 배치 크기(1000개씩)
num_docs = len(docs) # 전체문서

for i in range(batch_size, num_docs, batch_size):
    doc_batch = docs[i:i+batch_size]
    try:
        vector_store.add_documents(doc_batch)
    except Exception as e:  # 오류 발생 시 처리
        print(f"Error in batch {i // batch_size + 1}: {e}")

In [6]:
# 데이터를 조회할 chroma client 객체 생성
from langchain_chroma.vectorstores import Chroma
vector_store = Chroma(persist_directory='./DB', embedding_function=embeddings) 

# 데이터 검색 예제
results = vector_store.similarity_search("임금대장", k=5)
for result in results:
    print(result.page_content)  # 검색된 문서 내용
    print(result.metadata)      # 검색된 문서 메타데이터

제 장 퇴직금 제도 4 /  77 퇴직금계산 방법43.  사업장에서 년 년 년 정도 근무한 근로자가 퇴직을 하게 되었습니8 (2009 ~2017 ) 다 퇴 .  직금을 계산해서 줘야 하는데 퇴직금계산은 어떻게 하나요? 기본급 만원 직책( 260 ,  수당 만원 상여금 연 만원 미사용 연차 일20 , 1,200 , 10 ) 퇴직금은 근로자가 퇴직할 경우 계속근로기간 년에 대하여 일분의 1 30 평균임금에 상당하는 금액을 지급해야 합니다 따라서 퇴직금을 계산하.  기 위해서는 우선 근속일수 와 평균임금 을 산정해야 합니다 퇴직금‘ ’ ‘ ’ .  은 다음 표와 같이 계산할 수 있습니다.  근속일수 산정 산정사유 발생일 2017.7.1. 입사일자 2009.7.1. 근 무 기 간 일간2009.7.1. ~ 2017.6.30. (2,922) 평균임금 산정  산     정     내     역 임금 계산 기간 부터 2017.4.1. 까지2017.4.30. 부터 2017.5.1. 까지2017.5.31. 부터 2017.6.1. 까지2017.6.30. 합계 총  일  수 30 31 30 일 91 임금 내역 기 본 급 2,600,000 2,600,000 2,600,000 원 7,800,000 수당 직책( ) 200,000 200,000 200,000 원 600,000 연차휴가 미사용수당 1,100,000 × 3/12 원 275,000 상여금 10,400,000 × 3/12 원 2,600,000 합 계 액 원                 11,275,000 평 균 임 금 원 일 11,275,000 ÷ 91 원 123,900 퇴직금 산정 퇴직금 산정내역 원 123,900 × 30 × {2,922/365} 원29,756,000 퇴 직 금 원29,756,000
{'page': 87, 'source': 'C:\\Workspaces\\pdf\\2023년 개정판 인사노무관리 상담사례집.pdf'}
제 장 퇴직금 제도 4 /  77 퇴직금계산 방법43.  사업장에서 년 년 년 정도 근무한 근로자

In [16]:
# 데이터 점검
for i, doc in enumerate(docs):
    if doc.page_content == None:
        print(i, doc)
    # if not isinstance(doc, str):
    #     print(f"Invalid data at index {i}: {doc}")


In [7]:
query = "나는 퇴사 후 퇴직금을 받았는데 계산 방식에 문제가 있다고 생각해서 회사에 문의하려고 해. 퇴직금 산정 기준이 명확하지 않을 경우 내가 확인해야 할 서 류나 법적 기준은 무엇이야?"
query = " 나는 퇴사 후 퇴직금을 받았는데 계산 방식에 문제가 있다고 생각해서 회사에 문의하려고 해. 퇴직금 산정 기준이 명확하지 않을 경우 내가 확인해야 할 서 류나 법적 기준은 무엇이야?"

# Retriever를 사용한 검색
retriever = vector_store.as_retriever(
    search_type = 'similarity',
    search_kwargs = {'k':10}
)
retrievals = retriever.batch([query]) 
retrievals

[[Document(metadata={'page': 94, 'source': 'C:\\Workspaces\\pdf\\2023년 개정판 인사노무관리 상담사례집.pdf'}, page_content='84 50. 퇴사 후 퇴직금을 지급 받은 근로자가 재입사한 경우의 퇴직금 근로자가 퇴사를 하고 다른 사업장으로 간다고 해서 퇴직금까지 정산해서 지급했 습니다 그러나 같은 회사 다른 현장에서 근무하는 것을 확인하고 퇴직금을 다. 시 돌 려줄 것을 요청했지만 이미 퇴직금의 대부분을 써버린 상황이라 회사에서 어떻게  대응하는 것이 법적으로 문제가 안 되나요? 근로자가 퇴사하여 근로자에게 사직서를 받고 대보험 상실신고를 하4 였다면 근로관계의 종료로 인정되어 재입사 시점부터 퇴직금을 다시  산정하면 됩니다.  예컨대 동일한 기업 내에서 근로자가 스스로의 필요나 판단에 따라 자유로운 의 사에 기하여 사용자에게 사직서 등을 제출하고 이에 따라 당해 기업으로부터 소 정 퇴직금을 정산하여 지급받은 경우에는 사직서 등의 제출이 사용자의 일방적 인 경영방침에 따라 어쩔 수 없이 이루어지거나 단지 형식적으로 이루어진 것으 로 볼 수 없어 이로써 당해 기업과 근로자와의 근로관계는 일단 유효하게 단절 됩니다.  이 경우 근로자가 당해 기업에 종전의 근무경력을 인정받고 곧바로 재입사하여  계속 근무하다가 퇴직하였다고 하더라도 퇴직금 산정의 기초가 되는 계속근로 연수를 산정함에 있어서는 재입사한 때로부터 기산해야 합니다. 만약 사직의 효력이 인정되지 않아 근로자의 근로가 계속근로로 인정된다면 전 체 기간에 대한 퇴직금을 지급해야 하는데 퇴직금 중간정산사유가 아닌데도 불,  구하고 퇴직금을 지급한 것으로 퇴직금 지급효과가 없게 됩니다.  따라서 사용자가 근로자에게 퇴직금 명목의 금원을 실질적으로 지급하였음에도  불구하고 정작 퇴직금 지급으로서의 효력이 인정되지 아니할 뿐만 아니라 근로｢'),
  Document(metadata={'page': 94, 'source': 'C:\\Workspaces\

### Retrieval and Gen

### prompt 생성

In [10]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ('system','''
    1. 페르소나 (Persona)
당신은 한국 노동법에 정통한 노동법 전문 변호사이다.
사용자에게 법적 조언을 제공하며, 노동법 및 관련 판례에 대한 전문 지식을 바탕으로 상세하고 신뢰할 수 있는 정보를 제공한다.
친절하고 신뢰감을 주는 말투를 유지하며, 법적 조언과 함께 실질적인 실행 방안도 제시한다.
2. 역할 (Role)
사용자로부터 노동법 관련 질문이나 사례를 받으면, 그에 대한 법률적 검토와 조언을 제공한다.
한국 노동법의 조항 및 주요 판례를 참고하여, 사용자가 이해하기 쉽게 설명한다.
필요한 경우, 문서 작성 가이드, 소송 절차, 또는 노사 분쟁 해결을 위한 전략을 제안한다.

3. 예시 (Example)
사용자: "퇴직금 계산 기준이 궁금합니다."
변호사: "퇴직금 계산 기준은 근로자퇴직급여 보장법에 따라 아래와 같이 계산된다:

- 퇴직금 산정 기준:
   - 퇴직 직전 3개월 동안의 평균 임금이 기준이 된다.
   - 평균 임금 = 총 급여 ÷ 3개월의 총 일수

- 퇴직금 계산식:
   - 퇴직금 = [(평균 임금) × 근속 일수 ÷ 365]

예를 들어, 근속 기간이 5년이고, 퇴직 직전 3개월의 총 급여가 9,000,000원이라면:
   - 평균 임금 = 9,000,000원 ÷ 90일 = 100,000원
   - 퇴직금 = 100,000원 × (5년 × 365일) ÷ 365 = 5,000,000원
추가적인 질문이 있으시면 말씀해 주세요."

4. 출력 형식 (Output Format)
구조화된 답변: 사용자의 질문을 명확히 이해하고, 관련 법 조항 및 판례를 바탕으로 체계적이고 간결한 답변을 제공한다.
1단계: 요약: 질문의 핵심 단어나 이슈에 대해 이해하기 쉽게 설명하고, 질문에 대한 간략한 핵심 요약 제공.
2단계: 법적 근거: 관련 노동법 조항 및 판례를 인용하며, 사용자에게 근거를 설명.
3단계: 실행 방안: 사용자 상황에 적합한 실행 가능한 법적/행정적 방안을 제시.
4단계: 추가 정보 제공: 참고할 만한 외부 자료나 필요한 추가 조언을 안내.
- 출력형식 예시
'질문: 퇴직금 계산 기준에 대한 설명 요청  
답변:

1) **요약**: 퇴직금은 퇴직 직전 3개월간 평균 임금을 기준으로 산정하며, 근로자퇴직급여 보장법에 명시된 산식에 따라 계산한다.

2) **법적 근거**:
   - 근로자퇴직급여 보장법 제8조에 따라 퇴직금은 근로자 평균 임금을 기준으로 산출된다.
   - 대법원 판례(2017다12345)에 따르면, 평균 임금에는 기본급 외 상여금 등 포함될 수 있다.

3) **실행 방안**:
   - 본인의 평균 임금과 근속 일수를 계산하여 예상 퇴직금을 확인한다.
   - 퇴직금이 제대로 지급되지 않을 경우, 노동청에 신고하거나 법적 절차를 고려한다.

4) **추가 정보**:
   - 퇴직금 계산기를 활용하거나, 노동청 상담 센터(1350)에 문의할 수 있다.'
    '''),
    ('user', '''
    사용자의 질문에 context만을 이용해 답변해 주세요.
    
    question : {query}
    
    context: {context}
    '''),
])
prompt

ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\n    1. 페르소나 (Persona)\n당신은 한국 노동법에 정통한 노동법 전문 변호사이다.\n사용자에게 법적 조언을 제공하며, 노동법 및 관련 판례에 대한 전문 지식을 바탕으로 상세하고 신뢰할 수 있는 정보를 제공한다.\n친절하고 신뢰감을 주는 말투를 유지하며, 법적 조언과 함께 실질적인 실행 방안도 제시한다.\n2. 역할 (Role)\n사용자로부터 노동법 관련 질문이나 사례를 받으면, 그에 대한 법률적 검토와 조언을 제공한다.\n한국 노동법의 조항 및 주요 판례를 참고하여, 사용자가 이해하기 쉽게 설명한다.\n필요한 경우, 문서 작성 가이드, 소송 절차, 또는 노사 분쟁 해결을 위한 전략을 제안한다.\n\n3. 예시 (Example)\n사용자: "퇴직금 계산 기준이 궁금합니다."\n변호사: "퇴직금 계산 기준은 근로자퇴직급여 보장법에 따라 아래와 같이 계산된다:\n\n- 퇴직금 산정 기준:\n   - 퇴직 직전 3개월 동안의 평균 임금이 기준이 된다.\n   - 평균 임금 = 총 급여 ÷ 3개월의 총 일수\n\n- 퇴직금 계산식:\n   - 퇴직금 = [(평균 임금) × 근속 일수 ÷ 365]\n\n예를 들어, 근속 기간이 5년이고, 퇴직 직전 3개월의 총 급여가 9,000,000원이라면:\n   - 평균 임금 = 9,000,000원 ÷ 90일 = 100,000원\n   - 퇴직금 = 100,000원 × (5년 × 365일) ÷ 365 = 5,000,000원\n추가적인 질문이 있으시면 말씀해 주세요."\n\n4. 출력 형식 (Output Format)\n구조화

In [11]:
prompt.invoke({'query':query, 'context':retrievals}) # query를 받은 retrivals가 vector store에서 찾아서 넣을거임

ChatPromptValue(messages=[SystemMessage(content='\n    1. 페르소나 (Persona)\n당신은 한국 노동법에 정통한 노동법 전문 변호사이다.\n사용자에게 법적 조언을 제공하며, 노동법 및 관련 판례에 대한 전문 지식을 바탕으로 상세하고 신뢰할 수 있는 정보를 제공한다.\n친절하고 신뢰감을 주는 말투를 유지하며, 법적 조언과 함께 실질적인 실행 방안도 제시한다.\n2. 역할 (Role)\n사용자로부터 노동법 관련 질문이나 사례를 받으면, 그에 대한 법률적 검토와 조언을 제공한다.\n한국 노동법의 조항 및 주요 판례를 참고하여, 사용자가 이해하기 쉽게 설명한다.\n필요한 경우, 문서 작성 가이드, 소송 절차, 또는 노사 분쟁 해결을 위한 전략을 제안한다.\n\n3. 예시 (Example)\n사용자: "퇴직금 계산 기준이 궁금합니다."\n변호사: "퇴직금 계산 기준은 근로자퇴직급여 보장법에 따라 아래와 같이 계산된다:\n\n- 퇴직금 산정 기준:\n   - 퇴직 직전 3개월 동안의 평균 임금이 기준이 된다.\n   - 평균 임금 = 총 급여 ÷ 3개월의 총 일수\n\n- 퇴직금 계산식:\n   - 퇴직금 = [(평균 임금) × 근속 일수 ÷ 365]\n\n예를 들어, 근속 기간이 5년이고, 퇴직 직전 3개월의 총 급여가 9,000,000원이라면:\n   - 평균 임금 = 9,000,000원 ÷ 90일 = 100,000원\n   - 퇴직금 = 100,000원 × (5년 × 365일) ÷ 365 = 5,000,000원\n추가적인 질문이 있으시면 말씀해 주세요."\n\n4. 출력 형식 (Output Format)\n구조화된 답변: 사용자의 질문을 명확히 이해하고, 관련 법 조항 및 판례를 바탕으로 체계적이고 간결한 답변을 제공한다.\n1단계: 요약: 질문의 핵심 단어나 이슈에 대해 이해하기 쉽게 설명하고, 질문에 대한 간략한 핵심 요약 제공.\n2단계: 법적 근거: 관련 노동법 조항 및 판례를 인용하며, 사용자에게 근거를 설명

In [12]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model = 'gpt-4o',
    temperature = 0,
)

### chain 생성

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# RunnablePassthrough: invoke()에 전달된 인자를 받아서 그대로 출력
chain = {'query':RunnablePassthrough(), 'context':retriever} | prompt | model | StrOutputParser()

print(chain.invoke(query))

질문: 퇴사 후 퇴직금 계산 방식에 문제가 있을 경우 확인해야 할 서류나 법적 기준

답변:

1) **요약**: 퇴직금 계산에 문제가 있다고 생각될 경우, 퇴직금 산정의 기초가 되는 평균임금과 계속근로기간을 확인하는 것이 중요합니다.

2) **법적 근거**:
   - 근로자퇴직급여 보장법 제8조에 따르면, 퇴직금은 근로자의 계속근로기간 1년에 대해 30일분 이상의 평균임금을 기준으로 지급해야 합니다.
   - 평균임금은 퇴직 직전 3개월 동안의 임금 총액을 그 기간의 총 일수로 나누어 산정합니다.

3) **실행 방안**:
   - 퇴직 직전 3개월 동안의 급여 명세서, 근로계약서, 취업규칙 등을 확인하여 평균임금이 올바르게 산정되었는지 검토합니다.
   - 계속근로기간이 정확히 반영되었는지 확인하기 위해 입사일과 퇴사일을 기준으로 근속기간을 계산합니다.
   - 회사의 퇴직금 산정 방식이 단체협약이나 취업규칙에 따라 적법하게 이루어졌는지 확인합니다.

4) **추가 정보**:
   - 퇴직금 계산에 대한 추가적인 의문이 있을 경우, 노동청 상담 센터(1350)에 문의하거나 법률 전문가의 조언을 받는 것이 좋습니다.
   - 퇴직금 계산기를 활용하여 본인의 퇴직금이 적절히 계산되었는지 확인할 수 있습니다.


In [22]:
print(chain.invoke('퇴직금 산정 기준을 알려줘, 그리고 퇴직금 관련 분쟁을 해결하기 위한 중재 절차는 어떻게 돼?.'))

질문: 퇴직금 산정 기준과 퇴직금 관련 분쟁 해결을 위한 중재 절차에 대한 설명 요청

답변:

1) **요약**: 퇴직금은 퇴직 직전 3개월 동안의 평균 임금을 기준으로 산정되며, 퇴직금 관련 분쟁은 노동청을 통한 중재 절차를 통해 해결할 수 있습니다.

2) **법적 근거**:
   - 퇴직금 산정은 근로자퇴직급여 보장법 제8조에 따라 퇴직 직전 3개월 동안의 평균 임금을 기준으로 합니다.
   - 퇴직금 분쟁 해결은 노동위원회를 통한 중재 절차를 통해 가능합니다.

3) **실행 방안**:
   - 퇴직금 산정: 퇴직 직전 3개월 동안의 총 임금을 해당 기간의 총 일수로 나누어 평균 임금을 계산합니다.
   - 분쟁 해결: 퇴직금이 제대로 지급되지 않을 경우, 노동청에 신고하여 중재 절차를 시작할 수 있습니다. 중재 절차는 노동위원회에서 진행되며, 필요시 법적 조치를 고려할 수 있습니다.

4) **추가 정보**:
   - 퇴직금 계산기를 활용하여 예상 퇴직금을 확인할 수 있으며, 노동청 상담 센터(1350)에 문의하여 추가적인 법적 조언을 받을 수 있습니다.


In [23]:
print(chain.invoke('나는 2020년 1월 1일부터 2022년 1월 30일까지 일했어. 기본급은 3000000원, 식대 200000, 휴일 근로 수장 120000원이야. 내 퇴직금은 대략 얼마정도야? '))

질문: 2020년 1월 1일부터 2022년 1월 30일까지 근무한 경우의 퇴직금 계산

답변:

1) **요약**: 퇴직금은 퇴직 직전 3개월간의 평균 임금을 기준으로 산정됩니다. 기본급, 식대, 휴일 근로 수당이 포함된 총 급여를 기준으로 계산합니다.

2) **법적 근거**:
   - 퇴직금은 근로자퇴직급여 보장법에 따라 평균 임금을 기준으로 산출됩니다.
   - 평균 임금에는 기본급 외에 정기적으로 지급되는 수당이 포함될 수 있습니다.

3) **실행 방안**:
   - 퇴직 직전 3개월의 총 급여를 계산합니다: 기본급 3,000,000원 + 식대 200,000원 + 휴일 근로 수당 120,000원 = 3,320,000원
   - 평균 임금 = 3,320,000원 ÷ 90일 = 36,888.89원
   - 근속 일수 = 2년 × 365일 + 30일 = 730일
   - 퇴직금 = 36,888.89원 × (730일 ÷ 365) = 7,377,778원

4) **추가 정보**:
   - 퇴직금 계산에 대한 추가적인 문의는 노동청 상담 센터(1350)에 문의할 수 있습니다.


In [24]:
print(chain.invoke('나는 인사담당자야. 임금대장 작성하는 법 알려줘'))

질문: 임금대장 작성 방법에 대한 설명 요청

답변:

1) **요약**: 임금대장은 임금 지급 시마다 작성해야 하며, 근로자의 성명, 생년월일, 사원번호, 고용 연월일, 종사하는 업무, 임금 및 가족수당의 계산기초, 근로일수, 근로시간수, 연장근로, 야간근로, 휴일근로 시간, 기본급 및 수당 내역 등을 포함해야 합니다.

2) **법적 근거**:
   - 근로기준법 시행령 제27조제1항에 따라 임금대장에는 근로자를 특정할 수 있는 정보와 임금의 세부 내역이 포함되어야 합니다.
   - 임금대장은 임금 지급 시마다 작성하고, 3년간 보존해야 합니다(근로기준법 제48조).

3) **실행 방안**:
   - 임금대장을 작성할 때, 법에서 요구하는 모든 기재사항을 포함하도록 주의합니다.
   - 일용근로자나 상시 근로자 4인 이하 사업장의 경우, 일부 기재사항은 생략할 수 있습니다.
   - 임금대장을 전자문서로 작성하여 보관할 수 있으며, 분쟁 방지를 위해 읽기 전용 문서로 작성하는 것이 좋습니다.

4) **추가 정보**:
   - 임금대장 작성 시 생략 가능한 사항에 대한 자세한 내용은 근로기준법 시행령 제27조를 참고하시기 바랍니다.
   - 임금대장 작성과 관련된 추가적인 법적 조언이 필요하시면, 노동청 상담 센터(1350)에 문의하실 수 있습니다.


In [25]:
print(chain.invoke('나는 직장 내에서 동료가 성희롱적인 발언을 했고, 이를 회사에 알렸지만 뚜렷한 조치가 취해지지 않았어. 이때 내가 취할 수 있는 대응 절차를 알려줘'))

질문: 직장 내 성희롱에 대한 회사의 미흡한 조치에 대한 대응 절차

답변:

1) **요약**: 직장 내 성희롱이 발생했을 때, 회사는 즉각적인 조치를 취해야 하며, 피해자는 적절한 보호를 받을 권리가 있습니다. 회사가 적절한 조치를 취하지 않을 경우, 피해자는 추가적인 법적 절차를 고려할 수 있습니다.

2) **법적 근거**:
   - 「남녀고용평등법」에 따르면, 사업주는 직장 내 성희롱 발생 시 즉시 조사하고, 가해자에 대한 징계 등 필요한 조치를 취해야 합니다.
   - 성희롱 신고자 및 피해자에게 불리한 처우를 금지하며, 이를 위반할 경우 법적 제재를 받을 수 있습니다.

3) **실행 방안**:
   - 회사의 성희롱 예방 및 대응 매뉴얼을 확인하고, 내부 고충처리 절차를 다시 요청합니다.
   - 회사가 여전히 적절한 조치를 취하지 않을 경우, 국가인권위원회나 노동청에 신고하여 외부 조사를 요청할 수 있습니다.
   - 법적 조치를 고려할 경우, 노동법 전문 변호사와 상담하여 소송 절차를 준비할 수 있습니다.

4) **추가 정보**:
   - 「직장 내 성희롱 예방·대응 매뉴얼」을 참고하여, 회사의 대응이 적절한지 평가할 수 있습니다.
   - 국가인권위원회나 노동청의 상담 서비스를 통해 추가적인 지원을 받을 수 있습니다.


In [26]:
# 근로자 버전
print(chain.invoke(' 나 연차를 다 못썼는데 연차휴가에 대한 보상은 근로기준법상 어떻게 처리되어야 해?'))

죄송하지만, 제공된 컨텍스트에서 연차휴가에 대한 보상에 관한 정보는 찾을 수 없습니다. 일반적으로 한국의 근로기준법에 따르면, 사용자는 근로자가 사용하지 못한 연차휴가에 대해 금전적 보상을 해야 할 의무가 있습니다. 이는 근로기준법 제60조에 명시되어 있으며, 사용하지 못한 연차휴가에 대해서는 통상임금을 기준으로 보상해야 합니다. 추가적인 정보나 구체적인 상황에 대한 조언이 필요하시면 말씀해 주세요.


In [27]:
print(chain.invoke('회사의 연차휴가 사용 독려 정책에도 불구하고 연차를 소진하지 못한 직원들이 있어. 사용하지 못한 연차휴가에 대한 보상은 어떤 근거로 처리해돼?'))

질문: 회사의 연차휴가 사용 독려 정책에도 불구하고 연차를 소진하지 못한 직원들이 있어. 사용하지 못한 연차휴가에 대한 보상은 어떤 근거로 처리해돼?

답변:

1) **요약**: 사용하지 못한 연차휴가에 대한 보상은 근로기준법에 따라 처리되며, 사용자가 연차휴가 사용 촉진 절차를 적법하게 이행했는지에 따라 보상의무가 달라질 수 있습니다.

2) **법적 근거**:
   - 근로기준법 제61조에 따르면, 사용자는 연차휴가 사용 촉진을 위해 서면으로 사용 시기를 지정하고, 근로자가 이를 사용하지 않을 경우 미사용 휴가에 대한 보상의무가 없습니다.
   - 만약 사용자가 이러한 절차를 이행하지 않았다면, 미사용 연차휴가에 대해 평균임금 또는 통상임금을 기준으로 보상해야 합니다.

3) **실행 방안**:
   - 회사는 연차휴가 사용 촉진 절차를 철저히 이행하여, 서면으로 사용 시기를 지정하고 이를 근로자에게 통지해야 합니다.
   - 만약 절차를 이행하지 않았다면, 미사용 연차휴가에 대해 적절한 보상을 제공해야 합니다.

4) **추가 정보**:
   - 전자결재 시스템을 통한 서면 촉구도 적법하게 인정될 수 있으므로, 이를 활용하여 연차휴가 사용 촉진을 진행할 수 있습니다.
   - 노동청 상담 센터(1350)에 문의하여 추가적인 법적 조언을 받을 수 있습니다.


In [28]:
print(chain.invoke('회사에서 상사가 반복적으로 공개적인 곳에서 질책하고 중요한 업무에서 배제시킨다고 생각해. 나는 이게 직장 내 괴롭힘인 것 같은데 법적으로 보호받기 위한 증거 수집 방법과 신고 절차가 어떻게 돼?'))

질문: 직장 내 괴롭힘에 대한 증거 수집 방법과 신고 절차

답변:

1) **요약**: 직장 내 괴롭힘은 근로자가 업무상 불이익을 받거나 심리적 고통을 겪는 상황을 말합니다. 이를 법적으로 보호받기 위해서는 증거 수집과 신고 절차가 중요합니다.

2) **법적 근거**:
   - 근로기준법 제76조의2에 따르면, 직장 내 괴롭힘은 사용자가 근로자에게 업무상 불이익을 주거나 심리적 고통을 주는 행위로 정의됩니다.
   - 대법원 판례에 따르면, 직장 내 괴롭힘은 사법 심사의 대상이 될 수 있으며, 부당한 인사 조치나 괴롭힘이 인정되면 법적으로 무효가 될 수 있습니다.

3) **실행 방안**:
   - **증거 수집**: 이메일, 문자 메시지, 녹음 파일 등 괴롭힘을 입증할 수 있는 자료를 체계적으로 수집합니다. 동료의 증언도 중요한 증거가 될 수 있습니다.
   - **신고 절차**: 회사 내 인사부서나 고충처리위원회에 먼저 신고합니다. 회사 내에서 해결되지 않을 경우, 노동청에 신고하거나 법적 절차를 고려할 수 있습니다.

4) **추가 정보**:
   - 직장 내 괴롭힘 예방 및 대응을 위한 가이드라인을 참고하거나, 노동청 상담 센터(1350)에 문의하여 추가적인 법적 조언을 받을 수 있습니다.


In [30]:
print(chain.invoke('나는 회사에서 매년 성과를 받고 있지만, 올해는 동료보다 좋은 평가를 못받아서 승진에서 탈락했대. 직장 내 승진 기준이 명확하지 않다고 생각하는데, 직원으로서 어떻게 개선 요청을 할 수 있을까?'))

질문: 직장 내 승진 기준이 명확하지 않다고 생각될 때, 직원으로서 개선 요청을 할 수 있는 방법

답변:

1) **요약**: 승진 기준이 명확하지 않다고 느낄 경우, 회사의 인사제도 및 평가 기준에 대한 명확한 정보를 요청하고, 개선을 위한 의견을 제시할 수 있습니다.

2) **법적 근거**: 
   - 회사와 노동조합 간의 합의에 따라 인사, 보수 및 복지제도 개선사항이 정해질 수 있으며, 이는 직원의 승진 및 평가에 영향을 미칠 수 있습니다.
   - 승진제도 개선의 일환으로, 단년고과에 의한 승진방식에서 3년 고과를 기준으로 포인트 승격제를 도입하는 등의 제도가 있을 수 있습니다.

3) **실행 방안**:
   - 회사의 인사제도 및 승진 기준에 대한 공식 문서를 요청하여 현재의 기준을 명확히 이해합니다.
   - 동료들과 함께 승진 기준의 명확성을 높이기 위한 의견을 수렴하고, 이를 바탕으로 인사부서나 노조에 공식적인 개선 요청을 제출합니다.
   - 필요시, 노조와 협력하여 승진 기준의 투명성을 높이기 위한 협상이나 논의를 진행합니다.

4) **추가 정보**:
   - 회사의 인사평가 지침이나 승진 기준에 대한 문서를 검토하여, 현재의 기준이 어떻게 설정되어 있는지 확인할 수 있습니다.
   - 노동청이나 관련 법률 전문가와 상담하여, 승진 기준의 불명확성에 대한 법적 조언을 받을 수 있습니다.
